# Basic Reddit Scraping
Scraping from -> r/wallstreetbets
- Daily Discussion

In [1]:
!pip install praw
!pip install tqdm
!pip install transformers
!pip install ipywidgets
!pip install IProgress
!pip install Torch
!pip install BeautifulSoup4

  Using cached certifi-2021.5.30-py2.py3-none-any.whl (145 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pwntools 4.5.0 requires capstone>=3.0.5rc2, which is not installed.


Using legacy 'setup.py install' for get-all-tickers, since package 'wheel' is not installed.
    Running setup.py install for get-all-tickers: started
    Running setup.py install for get-all-tickers: finished with status 'done'


  Using cached PyYAML-5.4.1-cp38-cp38-win_amd64.whl (213 kB)


In [55]:
# Imports
import praw
import re
from praw.models import MoreComments
from datetime import datetime
import pandas as pd
import os
from tqdm import tqdm
from json import loads
from threading import Thread
from requests import Session
from bs4 import BeautifulSoup
from ast import literal_eval
import torch


In [56]:
cwd = os.path.abspath('')
print(cwd)

D:\Codes\Visual Studio\Projects\sentic-ai-project\datasets


In [57]:
config = loads(open(os.path.join(cwd, 'config/config.json')).read())
reddit_config = config["Reddit"]
reddit_config

{'user_agent': 'Comment Extraction (by u/SenticFinatic)',
 'client_id': 'JVpxM3nzVundq9JhLVOU3g',
 'client_secret': 'hH-hZ2VYqhLKNDc9tt0YHYAXvs1fdw',
 'username': 'senticfinatic',
 'password': 'Sentic{h!yDIi%GKhtExCT74iz5}'}

In [58]:
reddit = praw.Reddit(
    client_id=reddit_config["client_id"],
    client_secret=reddit_config["client_secret"],
    password=reddit_config["password"],
    user_agent=reddit_config["user_agent"],
    username=reddit_config["username"]
)

In [59]:
reddit.user.me()

Redditor(name='SenticFinatic')

# Pulling data from all wsb posts excluding memes
- From start to limit

## Pulling & Packing Needed Reddit Data(From Post Comments) into DataFrame

In [86]:
sub = next(reddit.subreddit("wallstreetbets").search("Daily Discussion"))
sub

Submission(id='prqy5j')

In [8]:
comments_list = list(sub.comments).copy()
body_comments_list = []
scores_comments_list = []
id_comments_list = []
time_comments_list = []
username_comments_list = []
replies_comments_list = []
sub.comments.replace_more(limit=0)


for comment in tqdm(sub.comments):
    if isinstance(MoreComments, type(comment)):
        continue
    body_comments_list.append(comment.body)
    scores_comments_list.append(comment.score)
    try:
        id_comments_list.append(comment.author.id)
    except:
#        print(comment.body)
        id_comments_list.append("Deleted")
    try:
        username_comments_list.append(comment.author.name)
    except:
        username_comments_list.append("Deleted")
    time_comments_list.append(datetime.utcfromtimestamp(comment.created_utc)\
                              .strftime('%Y-%m-%d %H:%M:%S'))
    replies_comments_list.append(len(comment.replies))
        
    

100%|█████████████████████████████████████████| 481/481 [06:06<00:00,  1.31it/s]


In [9]:
raw_dataset = pd.DataFrame({
    "Date": time_comments_list,
    "User ID": id_comments_list,
    "Username": username_comments_list,
    "Content": body_comments_list,
    "No. of Upvotes": scores_comments_list,
    "No. of Replies": replies_comments_list,
})


## Packing Reddit Data (From Reddit Posts) to DataFrame

In [118]:
POST_LIMIT = 500

In [119]:
sub = reddit.subreddit("wallstreetbets").new(limit=POST_LIMIT)
sub

In [120]:
raw_dataset = {
    "ID":[],
    "Date": [],
    "User ID": [],
    "Username": [],
    "Title":[],
    "Content": [],
    "Upvotes":[],
    "Comments":[]
}
properties = {
    "ID":"id",
    "Date": "created_utc",
    "User ID": "author.id",
    "Username": "author.name",
    "Title":"title",
    "Content": "selftext",
    "Upvotes": "score",
    "Comments": "comments"
}

In [121]:
for post in tqdm(sub, total=POST_LIMIT):
    for col in raw_dataset:
        try:
            if("author" in properties[col]):
                ppty = post
                for subppty in properties[col].split("."):
                    ppty = getattr(ppty, subppty)
            else:
                ppty = getattr(post, properties[col])
            raw_dataset[col].append(ppty)
        except Exception as e:
            raw_dataset[col].append("[Deleted]")
    
raw_dataset = pd.DataFrame(raw_dataset)

100%|█████████████████████████████████████████| 500/500 [14:00<00:00,  1.68s/it]


In [122]:
raw_dataset

,ID,Date,User ID,Username,Title,Content,Upvotes,Comments
0,ptqadj,1.632383e+09,qxk72,ElectroFried,Reported Evergande board meeting leak happenin...,,25,"(hdxurxw, hdxvbl2, hdxuxl3, hdxv0db, hdxv1gz, ..."
1,ptpqn3,1.632381e+09,175yyb,nosalute,Scientist discover why bears are the way they are,,38,"(hdxrqag, hdxs8zq, hdxu3b3, hdxvwm7)"
2,ptpq2s,1.632381e+09,jkktwg6,Namejamie,CLF IS STILL RED HOT,CLF or Cleveland-Cliffs is the largest flat ro...,6,"(hdxrn7z, hdxrpq6)"
3,ptpm6d,1.632380e+09,6wzaf14l,TensrWhoosh,Basically sums up most of retarded WSB apes,,11,"(hdxr30b, hdxr9s9, hdxrj4w, hdxskwl, hdxvzkq)"
4,ptpm19,1.632380e+09,6ed9i8cz,Irc-freak,$sofi gains. 2 hr hold.,,15,(hdxr1hp)
...,...,...,...,...,...,...,...,...
495,pqaxdb,1.631920e+09,3toea,space_cadet,Shorting China with $37k - YOLO,,2,"(hd9m1zy, hdjynkr, hdl65ug)"
496,pqawik,1.631920e+09,6mayhqwz,Geoffism1,"CLF was on sale today! $22,887 In Jan 2022 $25...",,38,"(hd9lw26, hd9pt4n, hd9s35v, hd9mai4, hd9paqm, ..."
497,pqaq4g,1.631919e+09,13a1wy,luckyrice,SoFi Basically All In Hoorah,,106,"(hd9kkmn, hd9lq7y, hda1k4z, hd9kvzy, hda4onz, ..."
498,pqamsd,1.631919e+09,a2tez47d,Johnny_Bravo_1,👀,,92,"(hd9jv0j, hd9kjom, hd9k92l, hd9o9ly, hd9kd87, ..."


In [123]:
raw_dataset.to_csv("raw_reddit_post_dataset.csv") 

In [8]:
raw_dataset = pd.read_csv("raw_reddit_post_dataset.csv")

# Data Cleaning (Using context parsing)

Instead of using this senticnet API for solely analysis, we can just context parsing to remove unnecessary data for ticker(stock symbol) filter

In [2]:
raw_dataset = pd.read_csv("raw_reddit_dataset.csv") # Loading previously scraped dataset
raw_dataset

,Unnamed: 0,Date,User ID,Username,Content,No. of Upvotes,No. of Replies
0,0,2021-09-20 20:16:31,4nc92,zjz,New Thread REEEEEEEEEEEEEEEEEEEEEEEEEEEE\n\nht...,1,0
1,1,2021-09-20 11:02:25,1293gu,picciona4,"Man, good thing those FED officials sold every...",226,0
2,2,2021-09-20 10:16:54,15tqn5,nutfugget,The Fed presidents disclosed they sold their s...,219,0
3,3,2021-09-20 11:44:01,kav2g,EatMaTesticles,This is the fifth once in a lifetime opportuni...,203,1
4,4,2021-09-20 11:12:21,2eptxwqx,dela_sole,Stupid evergrande blocking the river 🤦🏼‍♂️,152,1
...,...,...,...,...,...,...,...
476,476,2021-09-20 12:05:38,bz90j,Ruckefella,I got a great nights sleep last night lads. To...,11,0
477,477,2021-09-20 12:06:36,4fm5x,Shmoogy,The fact that everybody is so bearish ... i th...,11,0
478,478,2021-09-20 12:08:21,9xewy948,Fragrant-War-344,Dam happy I am holding 20% of my portfolio in ...,12,0
479,479,2021-09-20 12:10:43,i8sxc86,MakIkEenDonerMetKalf,The fed sold the top lmao,10,0


In [9]:
from transformers import pipeline

summarizer = pipeline("summarization")


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


In [10]:
def Summarise(dataset : pd.DataFrame, thread_num : int)-> pd.DataFrame:
    """
    Input:
    Dataset with Content Column
    Return:
    Dataset with modified Content Column
    """
    cleaned_ds = []
    for row in dataset.iloc():
        row["Content"] = summarizer(row["Content"])[0]["summary_text"]
        cleaned_ds.append(row)
    clean_ds[thread_num] = cleaned_ds

## Splitting Data

In [76]:
raw_dataset

,Unnamed: 0,ID,Date,User ID,Username,Title,Content,Upvotes,Comments
0,0,ptqadj,1.632383e+09,qxk72,ElectroFried,Reported Evergande board meeting leak happenin...,NaN,25,<praw.models.comment_forest.CommentForest obje...
1,1,ptpqn3,1.632381e+09,175yyb,nosalute,Scientist discover why bears are the way they are,NaN,38,<praw.models.comment_forest.CommentForest obje...
2,2,ptpq2s,1.632381e+09,jkktwg6,Namejamie,CLF IS STILL RED HOT,CLF or Cleveland-Cliffs is the largest flat ro...,6,<praw.models.comment_forest.CommentForest obje...
3,3,ptpm6d,1.632380e+09,6wzaf14l,TensrWhoosh,Basically sums up most of retarded WSB apes,NaN,11,<praw.models.comment_forest.CommentForest obje...
4,4,ptpm19,1.632380e+09,6ed9i8cz,Irc-freak,$sofi gains. 2 hr hold.,NaN,15,<praw.models.comment_forest.CommentForest obje...
...,...,...,...,...,...,...,...,...,...
495,495,pqaxdb,1.631920e+09,3toea,space_cadet,Shorting China with $37k - YOLO,NaN,2,<praw.models.comment_forest.CommentForest obje...
496,496,pqawik,1.631920e+09,6mayhqwz,Geoffism1,"CLF was on sale today! $22,887 In Jan 2022 $25...",NaN,38,<praw.models.comment_forest.CommentForest obje...
497,497,pqaq4g,1.631919e+09,13a1wy,luckyrice,SoFi Basically All In Hoorah,NaN,106,<praw.models.comment_forest.CommentForest obje...
498,498,pqamsd,1.631919e+09,a2tez47d,Johnny_Bravo_1,👀,NaN,92,<praw.models.comment_forest.CommentForest obje...


In [77]:
"""
In order to increase cleaning efficeny, utilise multithreading to
perform multiple requests at the same time to SenticNet API
"""
THREADS = 5
clean_ds = [None] * THREADS
def split_dataset(data, n):
    length = len(data)
    step = int(length / n)
    for x in range(step, length + 1, step):
        if length - x < step:
            yield data.iloc[x-step:]
        yield data.iloc[x-step:x]
     
#    for x in range(len(data)):
splitted_ds = list(split_dataset(raw_dataset, THREADS))
splitted_ds

[    Unnamed: 0      ID          Date   User ID        Username  \
 0            0  ptqadj  1.632383e+09     qxk72    ElectroFried   
 1            1  ptpqn3  1.632381e+09    175yyb        nosalute   
 2            2  ptpq2s  1.632381e+09   jkktwg6       Namejamie   
 3            3  ptpm6d  1.632380e+09  6wzaf14l     TensrWhoosh   
 4            4  ptpm19  1.632380e+09  6ed9i8cz       Irc-freak   
 ..         ...     ...           ...       ...             ...   
 95          95  ptaabz  1.632326e+09     ct8bp    leroyyrogers   
 96          96  pta59n  1.632325e+09  1chh8ec1  stellarzglitch   
 97          97  pt9wtp  1.632324e+09  4wjs17yo       Merkel420   
 98          98  pt9vzr  1.632324e+09  32s719y7       Kronodeus   
 99          99  pt9uu0  1.632324e+09   ffnhh1b       corradizo   
 
                                                 Title  \
 0   Reported Evergande board meeting leak happenin...   
 1   Scientist discover why bears are the way they are   
 2                  

In [78]:
sentic_config = config["SenticNet"]
CP_API = sentic_config["Concept_Parsing"]
CP_API

'z6S2XFFp4rZ1F'

In [86]:
def SenticNet(dataset : pd.DataFrame, API_KEY : str, targeted_col_name:str , new_col_name : str, thread_num:int, LANGUAGE:str ="en")->pd.DataFrame:
    """
    Require: 
    1) Dataset with ["Content"] column
    2) API Key for SenticNet API
    3) Targeted Column's name for passing content
    4) Column Name for data obtained from SenticNet API
    5) Thread Number (For storing returns)
    Returns:
    Appends Dataframe with [new_col_name : List] Column to returns variable (In order)
    """
    s = Session()
    url = f"https://sentic.net/api/{LANGUAGE}/{API_KEY}.py"
    new_col = []
    new_ds = dataset.copy() # For some reason the thing passed in is a view????, this is a temp fix, 
    for row in dataset.iloc():
        params = {
            "text" : row[targeted_col_name].replace(" ", "%20")
        }
        res = s.get(url, params=params)
        new_col.append(res.text)
    new_ds[new_col_name] = new_col
    returns[thread_num] = new_ds


In [87]:
# Sentic Concept Parse
returns = [None] * THREADS
ts = [None] * THREADS
for thread_num in range(THREADS):
    ts[thread_num] = Thread(target=SenticNet, args=(splitted_ds[thread_num], CP_API, "Title", "Concepts", thread_num))
    ts[thread_num].start()
for x in tqdm(range(THREADS)):
    ts[x].join()


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:52<00:00, 10.42s/it]


In [89]:
new_ds = pd.concat(returns)
new_ds

,Unnamed: 0,ID,Date,User ID,Username,Title,Content,Upvotes,Comments,Concepts
0,0,ptqadj,1.632383e+09,qxk72,ElectroFried,Reported Evergande board meeting leak happenin...,NaN,25,<praw.models.comment_forest.CommentForest obje...,"['board', 'leak', 'rip', 'bond']\n"
1,1,ptpqn3,1.632381e+09,175yyb,nosalute,Scientist discover why bears are the way they are,NaN,38,<praw.models.comment_forest.CommentForest obje...,['discover']\n
2,2,ptpq2s,1.632381e+09,jkktwg6,Namejamie,CLF IS STILL RED HOT,CLF or Cleveland-Cliffs is the largest flat ro...,6,<praw.models.comment_forest.CommentForest obje...,['red hot']\n
3,3,ptpm6d,1.632380e+09,6wzaf14l,TensrWhoosh,Basically sums up most of retarded WSB apes,NaN,11,<praw.models.comment_forest.CommentForest obje...,"['basically', 'retarded']\n"
4,4,ptpm19,1.632380e+09,6ed9i8cz,Irc-freak,$sofi gains. 2 hr hold.,NaN,15,<praw.models.comment_forest.CommentForest obje...,"['gains', 'hold']\n"
...,...,...,...,...,...,...,...,...,...,...
495,495,pqaxdb,1.631920e+09,3toea,space_cadet,Shorting China with $37k - YOLO,NaN,2,<praw.models.comment_forest.CommentForest obje...,No concepts detected\n
496,496,pqawik,1.631920e+09,6mayhqwz,Geoffism1,"CLF was on sale today! $22,887 In Jan 2022 $25...",NaN,38,<praw.models.comment_forest.CommentForest obje...,['sale']\n
497,497,pqaq4g,1.631919e+09,13a1wy,luckyrice,SoFi Basically All In Hoorah,NaN,106,<praw.models.comment_forest.CommentForest obje...,['basically']\n
498,498,pqamsd,1.631919e+09,a2tez47d,Johnny_Bravo_1,👀,NaN,92,<praw.models.comment_forest.CommentForest obje...,No concepts detected\n


In [41]:
threads = []
for x in range(THREADS):
    threads.append(Thread(target=Summarise,args=(splitted_ds[x],x,)))
print(threads)
for x in range(THREADS):
    threads[x].start()
for x in range(THREADS):
    threads[x].join()
clean_ds = pd.DataFrame(clean_ds)
print("Completed")
clean_ds


Your max_length is set to 142, but you input_length is only 57. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 13. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your min_length is set to 56, but you input_length is only 21. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your min_length is set to 56, but you input_length is only 7. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your min_length is set to 56, but you input_length is only 12. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 13. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 21. You might consider decr

[<Thread(Thread-19, initial)>, <Thread(Thread-20, initial)>, <Thread(Thread-21, initial)>, <Thread(Thread-22, initial)>, <Thread(Thread-23, initial)>]


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
Your min_length is set to 56, but you input_length is only 19. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 19. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 27. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 27. You might consider decreasing max_length manually, e.g. summarizer('...

Your min_length is set to 56, but you input_length is only 17. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 17. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 20. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 20. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 24. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 24. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 11. You might consider dec

Your max_length is set to 142, but you input_length is only 29. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 15. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 15. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 23. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 23. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 30. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 75. You might consider d

...

Your max_length is set to 142, but you input_length is only 19. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 12. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 12. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 15. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 15. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 6. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 6. You might consider decr

Your max_length is set to 142, but you input_length is only 24. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 22. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 22. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 6. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 6. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 13. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 13. You might consider decr

Your max_length is set to 142, but you input_length is only 49. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 24. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 24. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 40. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 45. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 21. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 21. You might consider d

Your max_length is set to 142, but you input_length is only 12. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 6. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 6. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 37. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 49. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 23. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 23. You might consider dec

Completed


# Harvesting off ticker counts without sentiment analysis
- In sentiment analysis, we would give a score for the top 5 root comments of the comment/post rrelated to the position
- This ticker counter does not include that

In [9]:
def get_stock_tickers():
    s = Session()
    url = "https://api.stockanalysis.com/wp-json/sa/search?q=index"
    stocks = literal_eval(s.get(url).text)
    tickers = {
        "ticker":[],
        "Name":[],
    }
    for stock in tqdm(stocks):
        tickers["ticker"].append(stock["s"])
        tickers["Name"].append(stock["n"])
    return tickers
tickers = pd.DataFrame(get_stock_tickers())
tickers
    

100%|█████████████████████████████████████████████████████████████████████████| 9437/9437 [00:00<00:00, 3147896.20it/s]


,ticker,Name
0,A,Agilent Technologies
1,AA,Alcoa
2,AAC,Ares Acquisition
3,AACE,Alexandria Agtech\/Climate Innovation
4,AACG,ATA Creativity Global
...,...,...
9432,ZHDG,ZEGA Buy and Hedge ETF
9433,ZIG,Acquirers Fund
9434,ZJPN,SPDR Solactive Japan ETF
9435,ZROZ,PIMCO 25 Plus Year Zero Coupon U.S. Treasury I...


In [10]:
tickers.to_csv("tickers.csv")

In [50]:
raw_dataset

,Unnamed: 0,ID,Date,User ID,Username,Title,Content,Upvotes,Comments
0,0,ptqadj,1.632383e+09,qxk72,ElectroFried,Reported Evergande board meeting leak happenin...,NaN,25,<praw.models.comment_forest.CommentForest obje...
1,1,ptpqn3,1.632381e+09,175yyb,nosalute,Scientist discover why bears are the way they are,NaN,38,<praw.models.comment_forest.CommentForest obje...
2,2,ptpq2s,1.632381e+09,jkktwg6,Namejamie,CLF IS STILL RED HOT,CLF or Cleveland-Cliffs is the largest flat ro...,6,<praw.models.comment_forest.CommentForest obje...
3,3,ptpm6d,1.632380e+09,6wzaf14l,TensrWhoosh,Basically sums up most of retarded WSB apes,NaN,11,<praw.models.comment_forest.CommentForest obje...
4,4,ptpm19,1.632380e+09,6ed9i8cz,Irc-freak,$sofi gains. 2 hr hold.,NaN,15,<praw.models.comment_forest.CommentForest obje...
...,...,...,...,...,...,...,...,...,...
495,495,pqaxdb,1.631920e+09,3toea,space_cadet,Shorting China with $37k - YOLO,NaN,2,<praw.models.comment_forest.CommentForest obje...
496,496,pqawik,1.631920e+09,6mayhqwz,Geoffism1,"CLF was on sale today! $22,887 In Jan 2022 $25...",NaN,38,<praw.models.comment_forest.CommentForest obje...
497,497,pqaq4g,1.631919e+09,13a1wy,luckyrice,SoFi Basically All In Hoorah,NaN,106,<praw.models.comment_forest.CommentForest obje...
498,498,pqamsd,1.631919e+09,a2tez47d,Johnny_Bravo_1,👀,NaN,92,<praw.models.comment_forest.CommentForest obje...


In [91]:
filtered_dataset = {
    "Comment/Post": [],
    "Corresponding ticker": []
}
patterns = {ticker:re.compile(f"\W{ticker}\W") for ticker in tickers["ticker"].iloc()}
patterns

ds = new_ds
col = "Title" # Col with content to find/extract tickers
for x in tqdm(range(len(ds[col]))):
    tkers = []
    for ticker in patterns:
        results = re.search(patterns[ticker], ds[col][x])
        if results:
            tkers.append(ticker)
    if len(tkers) != 0:
        filtered_dataset["Comment/Post"].append(ds.iloc[x])
        filtered_dataset["Corresponding ticker"].append(tkers)


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:24<00:00, 20.20it/s]


In [92]:
fd = pd.DataFrame(filtered_dataset["Comment/Post"])
fd["Stock"] = filtered_dataset["Corresponding ticker"]

In [23]:
tickers

,ticker,Name
0,A,Agilent Technologies
1,AA,Alcoa
2,AAC,Ares Acquisition
3,AACE,Alexandria Agtech\/Climate Innovation
4,AACG,ATA Creativity Global
...,...,...
9432,ZHDG,ZEGA Buy and Hedge ETF
9433,ZIG,Acquirers Fund
9434,ZJPN,SPDR Solactive Japan ETF
9435,ZROZ,PIMCO 25 Plus Year Zero Coupon U.S. Treasury I...


In [93]:
a = []
analytics = {
    "data": {ticker:[] for ticker in tickers["ticker"]}
}
MIN_POSTS = 3
for x in range(len(filtered_dataset["Corresponding ticker"])):
    for y in filtered_dataset["Corresponding ticker"][x]:
        for ticker in analytics["data"]:
            if ticker == y:
                analytics["data"][ticker].append(fd.iloc[x])
                break

analytics["data"] = {ticker:pd.DataFrame(analytics["data"][ticker]) for ticker in analytics["data"]}
popped_tickers = []
for key in analytics["data"]:
    if len(analytics["data"][key]) < MIN_POSTS:
        popped_tickers.append(key)
for ticker in popped_tickers:
    analytics["data"].pop(ticker)
analytics["sentiment"] =  {ticker:0 for ticker in analytics["data"]} # Creating only tickers that have related posts
analytics

{'data': {'A':      Unnamed: 0      ID          Date   User ID           Username  \
  43           43  pti1rs  1.632352e+09     u84x4           mactech3   
  287         287  ps5fto  1.632177e+09  ebmb73rj  Turbulent_Dog_628   
  376         376  prtrge  1.632141e+09  89a87wr2       Order_Flimsy   
  384         384  prqtr6  1.632128e+09   odla3qg          BIRBIGD99   
  451         451  pqqt3r  1.631987e+09  a0kz65xr        oneenigma4u   
  455         455  pqpq27  1.631983e+09     g0xvh          Berisha11   
  
                                                   Title  \
  43   $4 Million BRK.A & B Position - Best Stock in ...   
  287  The Big Dipper: A bull case for GREE, the #1 m...   
  376        Could Bumble Help You Retire A Millionaire?   
  384  China is fucked. So lets make money. (ASHR, EW...   
  451  [GME & AMC ] A SIMPLE EXPLANATION WHY THEY TOO...   
  455  How $SPY puts made me find the woman of my dre...   
  
                                                 Content 

In [52]:
analytics["data"]["SPY"]

,Unnamed: 0,ID,Date,User ID,Username,Title,Content,Upvotes,Comments,Stock
19,19,ptmnsj,1.632368e+09,a15t43vb,wsbmuskrat,"A friend, Never bet against America. $SPY PUTS",NaN,1248,<praw.models.comment_forest.CommentForest obje...,[SPY]
20,20,ptmh5d,1.632367e+09,1onht8u1,atac03,"SOFI, ATVI, EA, SPY, WFC YOLO. VERY dIvERsiFIE...",NaN,15,<praw.models.comment_forest.CommentForest obje...,"[ATVI, EA, VERY, WFC, SPY, YOLO]"
36,36,ptjheu,1.632357e+09,13bph0,cashbackpal,"Got rekt by YANG today (-5k), day traded my wa...",NaN,36,<praw.models.comment_forest.CommentForest obje...,"[SPY, YANG]"
108,108,pt8av8,1.632320e+09,5x9cxmk5,globohydrate,26k SPY 0DTE yolo,NaN,32,<praw.models.comment_forest.CommentForest obje...,[SPY]
186,186,psr7ds,1.632256e+09,9xkvswtl,rebelo55,$SPY today,NaN,13569,<praw.models.comment_forest.CommentForest obje...,[SPY]
204,204,pso9zs,1.632247e+09,4dwj5987,OTMorDeath,Update from mini SPY YOLO (if anyone cares) so...,NaN,27,<praw.models.comment_forest.CommentForest obje...,"[SPY, YOLO]"
237,237,psjp2c,1.632234e+09,a3gcdrz9,BFLO-Retail,How to short SPY with Synthetic Short Shares C...,What are SPY synthetic short shares?? (and why...,12,<praw.models.comment_forest.CommentForest obje...,[SPY]
269,269,psb5h0,1.632197e+09,l2i7wdb,Mcdaddybigballs602,Sportsbook doesn’t work like $SPY calls.,NaN,31,<praw.models.comment_forest.CommentForest obje...,[SPY]
285,285,ps5yav,1.632179e+09,5vou12u8,ROCKETWAE,People with SPY calls right now,NaN,527,<praw.models.comment_forest.CommentForest obje...,[SPY]
290,290,ps56w7,1.632176e+09,w5brib,mpoozd,This is why SPY skyrocketed in the last 30 min...,NaN,399,<praw.models.comment_forest.CommentForest obje...,[SPY]


# Average Sentiment Analysis
- Preparation
    - We utilise SenticNet's API to extract concepts that are meaningful for Sentiment Analysis
- Evaluation
    - Evalate a sentiment score for each individual post
    - Calculate average sentiment score for all posts

In [94]:
from transformers import pipeline
sentiment = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


In [95]:
col = "Concepts" # Analyse by title
def sentiment_analysis(content):
    datas = sentiment(content)
    total = 0
    for data in datas:
        if(data["label"] == "NEGATIVE"):
            total += -data["score"] 
        else:
            total += data["score"]
    if len(data) != 0:
        return total / len(data)
    else:
        return 0

for ticker in analytics["data"]:
    score = 0
    analytics["data"][ticker]["score"] = analytics["data"][ticker][col].apply(sentiment_analysis)
    analytics["sentiment"][ticker] = analytics["data"][ticker]["score"].sum() / analytics["data"][ticker]["score"].count()
    
    

In [96]:
analytics["sentiment"]

{'A': -0.46044722696145374,
 'AMC': -0.195493683218956,
 'ASTS': -0.33470456089292255,
 'BABA': 0.166407972574234,
 'BB': 0.33805980001177105,
 'CLF': -0.16671531399091086,
 'DD': 0.21416022947856359,
 'GME': -0.14847712218761444,
 'IRNT': 0.09727525285312108,
 'LCID': -0.12456714113553365,
 'OPAD': -0.4406326711177826,
 'S': -0.18132385185786656,
 'SDC': 0.09991996884346008,
 'SPIR': -0.25175125896930695,
 'TMC': -0.3136305255549295,
 'U': 0.1636685828367869,
 'UPST': -0.05320754647254944,
 'WISH': 0.24951542541384697,
 'SPY': -0.27078565359115603,
 'YANG': -0.1631395717461904,
 'YOLO': 0.022546873010438066}

In [97]:
analytics.to_csv("analytics.csv")

AttributeError: 'dict' object has no attribute 'to_csv'